In [1]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [2]:
print(train_df.shape)
print(test_df.shape)

(891, 12)
(418, 11)


In [3]:
# isna, sum: train_df.isna().sum()
s = train_df.isna().sum()
s[s > 0]

Age         177
Cabin       687
Embarked      2
dtype: int64

In [4]:
# isna, sum: train_df.isna().sum()
s = test_df.isna().sum()
s[s > 0]

Age       86
Fare       1
Cabin    327
dtype: int64

In [5]:
# 數值類型缺失直: 補中位數
# medain(), mean()
# Pclass: 其實是類別, 不該填中位數
med = train_df.median().drop(["Pclass"])
train_df = train_df.fillna(med)
# test要不要重算一次med? 不能重算!!
test_df = test_df.fillna(med)
# isna, sum: train_df.isna().sum()
s = test_df.isna().sum()
s[s > 0]

Cabin    327
dtype: int64

In [6]:
# value_counts/idxmax
# 類別型態補缺失職: 捕最常出現
most = train_df["Embarked"].value_counts().idxmax()
train_df["Embarked"] = train_df["Embarked"].fillna(most)
s = train_df.isna().sum()
s[s > 0]

Cabin    687
dtype: int64

In [19]:
n = train_df["Name"][0]
n = n.split(",")[-1].split(".")[0]
n.strip()
# import re
# (\w+): 所有的a-z0-9, 名字
# \s*: match出現一次以上的空白
# (\w+): 中間稱謂
# \s*: match姓氏前面的空白
# ([\w\s]+): match姓氏, 姓氏之間是有空白
# pat = r"(\w+),\s*(\w+)\.\s*([\w\s]+)"
# re.search(pat, n).group(3)

'Mr'

In [0]:
def middle(n):
    n = n.split(",")[-1].split(".")[0]
    return n.strip()
train_df["Name"].apply(middle).value_counts()

In [21]:
def middle(n):
    reserved = ["Mr", "Mrs", "Miss", "Master"]
    n = n.split(",")[-1].split(".")[0]
    if n.strip() in reserved:
        return n.strip()
    else:
        return None
train_df["Name"] = train_df["Name"].apply(middle)
train_df["Name"].value_counts()

Mr        517
Miss      182
Mrs       125
Master     40
Name: Name, dtype: int64